# **Case studies**$\def\ket#1{\left|#1\right\rangle}\def\bra#1{\left\langle #1\right|}$

In [1]:
import sys
sys.path.insert(0, "../..") # clue is here

from clue.qiskit import *
from math import log

In this document we present examples of the lumping on quantum circuits from the *draft*. In particular, we will present what it means a lumping on several problems and how to extract the information directly from the lumping. The problems we are going to focus in this document are:

* The **Search algorithm** using Grover's gate.
* The **Order computation** using a multiplication by $x$ module $N$ gate.
* The **Phase estimation** problem using Kitaev's gate based on the multiplication by $x$ module $N$ gate.

## 1. *Search algorithm*

Let $f: \{0,\ldots,2^n-1\} \rightarrow \{0,1\}$ be a boolean function that defines a success criteria for a search. Grover's gate is defined by:
$$G \ket{x} = (-1)^{f(x)} \left(I - 2 \ket{\psi} \bra{\psi}\right)\ket{x}.$$

In the *draft* we shows that the unitary matrix that defines Grover's gate enjoys a lumping when observing the entangled state $\ket{\psi}$. More precisely, since the entangled state can be represented as a linear combination of the *success* state and *failure* state, then CLUE will return a lumping of length 2 that will contain all the information about these two states.

In [2]:
f = lambda p : 0 if p == 0 else 1 if 3**(int(log(p)/log(3))) == p else 0 # looks for powers of 3
print(f"Success search: {[i for i in range(1,256) if f(i) == 1]}")
G(f,4)

Success search: [1, 3, 9, 27, 81]


array([[ 0.875+0.j,  0.125-0.j, -0.125+0.j,  0.125-0.j, -0.125+0.j,
        -0.125+0.j, -0.125+0.j, -0.125+0.j, -0.125+0.j,  0.125-0.j,
        -0.125+0.j, -0.125+0.j, -0.125+0.j, -0.125+0.j, -0.125+0.j,
        -0.125+0.j],
       [-0.125+0.j, -0.875+0.j, -0.125+0.j,  0.125-0.j, -0.125+0.j,
        -0.125+0.j, -0.125+0.j, -0.125+0.j, -0.125+0.j,  0.125-0.j,
        -0.125+0.j, -0.125+0.j, -0.125+0.j, -0.125+0.j, -0.125+0.j,
        -0.125+0.j],
       [-0.125+0.j,  0.125-0.j,  0.875+0.j,  0.125-0.j, -0.125+0.j,
        -0.125+0.j, -0.125+0.j, -0.125+0.j, -0.125+0.j,  0.125-0.j,
        -0.125+0.j, -0.125+0.j, -0.125+0.j, -0.125+0.j, -0.125+0.j,
        -0.125+0.j],
       [-0.125+0.j,  0.125-0.j, -0.125+0.j, -0.875+0.j, -0.125+0.j,
        -0.125+0.j, -0.125+0.j, -0.125+0.j, -0.125+0.j,  0.125-0.j,
        -0.125+0.j, -0.125+0.j, -0.125+0.j, -0.125+0.j, -0.125+0.j,
        -0.125+0.j],
       [-0.125+0.j,  0.125-0.j, -0.125+0.j,  0.125-0.j,  0.875+0.j,
        -0.125+0.j, -0.125+0.j, 

We can apply CLUE to this matrix and the observable given by $\ket{\psi}$:

In [7]:
system = DS_QuantumCircuit(G(f,4))
lumped = system.lumping([SparsePolynomial.from_vector(Psi(2**4), system.variables, system.field)])

New variables:
y0 = (0.25 + 0.0j)*Q_0000 + (0.25 + 0.0j)*Q_0001 + (0.25 + 0.0j)*Q_0010 + (0.25 + 0.0j)*Q_0011 + (0.25 + 0.0j)*Q_0100 + (0.25 + 0.0j)*Q_0101 + (0.25 + 0.0j)*Q_0110 + (0.25 + 0.0j)*Q_0111 + (0.25 + 0.0j)*Q_1000 + (0.25 + 0.0j)*Q_1001 + (0.25 + 0.0j)*Q_1010 + (0.25 + 0.0j)*Q_1011 + (0.25 + 0.0j)*Q_1100 + (0.25 + 0.0j)*Q_1101 + (0.25 + 0.0j)*Q_1110 + (0.25 + 0.0j)*Q_1111
y1 = (-0.120096115353815 + 0.0j)*Q_0000 + (0.520416499866533 + 0.0j)*Q_0001 + (-0.120096115353815 + 0.0j)*Q_0010 + (0.520416499866533 + 0.0j)*Q_0011 + (-0.120096115353815 + 0.0j)*Q_0100 + (-0.120096115353815 + 0.0j)*Q_0101 + (-0.120096115353815 + 0.0j)*Q_0110 + (-0.120096115353815 + 0.0j)*Q_0111 + (-0.120096115353815 + 0.0j)*Q_1000 + (0.520416499866533 + 0.0j)*Q_1001 + (-0.120096115353815 + 0.0j)*Q_1010 + (-0.120096115353815 + 0.0j)*Q_1011 + (-0.120096115353815 + 0.0j)*Q_1100 + (-0.120096115353815 + 0.0j)*Q_1101 + (-0.120096115353815 + 0.0j)*Q_1110 + (-0.120096115353815 + 0.0j)*Q_1111
New initial conditions

As expected, we got 2 macro-states. One of them is $\ket{\psi}$ itself. We should be able to obtain the success and failure states from these two macro-states:

In [19]:
L = lumped.lumping_matrix.to_numpy(dtype=cdouble)
## Computing the reduced echelon form
new_L = [L[0],L[1]]
new_L[1] = L[1] - L[0]*L[1][0]/L[0][0] # creating zeros on the second row
first_nonzero = min([i for i in range(len(new_L[1])) if new_L[1][i] != 0])
new_L[0] = new_L[0] - new_L[1]*new_L[0][first_nonzero]/new_L[1][first_nonzero]
## Comuting the normalized version
new_L = [row/(sqrt(inner_product(row,row))) for row in new_L]
new_L = array([new_L[0],new_L[1]], dtype=cdouble)
new_L

array([[0.2773501 +0.j, 0.        +0.j, 0.2773501 +0.j, 0.        +0.j,
        0.2773501 +0.j, 0.2773501 +0.j, 0.2773501 +0.j, 0.2773501 +0.j,
        0.2773501 +0.j, 0.        +0.j, 0.2773501 +0.j, 0.2773501 +0.j,
        0.2773501 +0.j, 0.2773501 +0.j, 0.2773501 +0.j, 0.2773501 +0.j],
       [0.        +0.j, 0.57735027+0.j, 0.        +0.j, 0.57735027+0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.57735027+0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j]])

In [23]:
## checking the states
states = [[i for i in range(len(row)) if row[i] != 0] for row in new_L]
states

[[0, 2, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15], [1, 3, 9]]

We can see here that thes are the two states: the failure state (first row) and the success state (second row). Once we have splitted the two cases, we could check which one is the failure and the success by evaluating once the oracle $f(\cdot).$

In [24]:
[f(state[0]) for state in states] 

[0, 1]

## 2. *Order finding*

Let $x \in \{2,3,\ldots, N-1\}$. We want to find the order of $x$ module $N$, i.e., find the minimal $r$ such that $x^r = 1 (mod\ N)$. Using the results from the *draft*, we know that for a gate that permorms the multiplication by $x$ module $N$ enjoys a lumping when setting the observable $\ket{1}$. More precisely, the dimension of the lumpin es exactly $r$.

In [47]:
system = DS_QuantumCircuit(U(7,32))
lumped = system.lumping([SparsePolynomial.from_vector([0,1] + (system.size-2)*[0], system.variables, system.field)])

New variables:
y0 = Q_00001
y1 = Q_10111
y2 = Q_10001
y3 = Q_00111
New initial conditions:
Lumped system:
y0' = y1
y1' = y2
y2' = y3
y3' = y0


In [48]:
7**4 % 32

1

## 3. *Phase Estimation*

In this problem, we are assume that we know a quantum gate $U$ that has a known eigenvector $u$. Since we are working with quantum states and the matrices are unitary, this means that the eigenvalues must be complex numbers of modulus one. Hence, they are of the form $e^{2\pi i \phi}$, where $\phi$ is called the phase of the eigenvalue.

In this problem we try to estimate with $n$ bits the phase $\phi$ by writing 
$$\tilde{\phi} = 0.\phi_{n-1}\cdots\phi_0 = \frac{\phi_{n-1}}{2} + \ldots + \frac{\phi_0}{2^n}.$$

Kitaev circuit is the case for estimating $\phi_{n-1}$, it can be reused to obtain further bits or it can be recombined as in Figure 5.3 (see Nielsen-CHuang book) to obtain all bits at once. In the *draft* , we discuss two lumping cases: the first is when taking $\ket{j}\ket{u}$, obtaining a lumping of size 2 (i.e., we can simulate $K$ with 1 q-bit). The second case is when we consider the entangled state
$$\left[\frac{\ket{0} - \ket{1}}{2}\right]\ket{u}$$

Let us fix $U$ to be defined as in Section 2 (i.e., the circuit that multiplies numbers $y \in [0,\ldots,N-1]$ by a fixed number $x$:

In [2]:
x = 7; N = 32
U(x,N)

array([[1.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       ...,
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]])

These are the eigenvectors for this matrix:

In [3]:
from numpy.linalg  import eig
_, vects = eig(U(7,32)); vects

array([[-0.        -0.j, -0.        -0.j, -0.        -0.j, ...,
         0.        +0.j,  1.        +0.j,  0.        +0.j],
       [-0.        -0.j, -0.        -0.j, -0.        -0.j, ...,
         0.        +0.j,  0.        +0.j,  0.        +0.j],
       [-0.        -0.j, -0.        -0.j,  0.70710678-0.j, ...,
         0.        +0.j,  0.        +0.j,  0.        +0.j],
       ...,
       [-0.        -0.j, -0.        -0.j, -0.        -0.j, ...,
         0.        +0.j,  0.        +0.j,  0.        +0.j],
       [ 0.70710678-0.j, -0.70710678-0.j, -0.        -0.j, ...,
         0.        +0.j,  0.        +0.j,  0.        +0.j],
       [-0.        -0.j, -0.        -0.j, -0.        -0.j, ...,
         0.        +0.j,  0.        +0.j,  0.        +0.j]])

Let us consider the following eigenvector

In [4]:
u = vects[:,2]; u

array([-0.        -0.j, -0.        -0.j,  0.70710678-0.j, -0.        -0.j,
       -0.        -0.j, -0.        -0.j, -0.        -0.j, -0.        -0.j,
       -0.        -0.j, -0.        -0.j, -0.        -0.j, -0.        -0.j,
       -0.        -0.j, -0.        -0.j,  0.70710678+0.j, -0.        -0.j,
       -0.        -0.j, -0.        -0.j, -0.        -0.j, -0.        -0.j,
       -0.        -0.j, -0.        -0.j, -0.        -0.j, -0.        -0.j,
       -0.        -0.j, -0.        -0.j, -0.        -0.j, -0.        -0.j,
       -0.        -0.j, -0.        -0.j, -0.        -0.j, -0.        -0.j])

Now, let's see if we can compute the lumpings with this eigenvector:

In [9]:
system = DS_QuantumCircuit(K(U(x,N)))
obs = [SparsePolynomial.from_vector(kron([1,0], u), system.variables, CC)] # |0> |u>

In [10]:
lumped = system.lumping(obs)

New variables:
y0 = (0.707106781186547 + 0.0j)*Q_000010 + (0.707106781186548 + 0.0j)*Q_001110
New initial conditions:
Lumped system:
y0' = y0
